In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os 
from openai import OpenAI
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec



In [ ]:

pc = Pinecone(api_key='yoir-api-key')
pc.create_index(
        name='rag',
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
)

In [ ]:
import json 
data = json.load(open('reviews.json'))
data['reviews']

In [ ]:
processed_data = []
client = OpenAI(api_key='your key')

for review in data['reviews']:
   response = client.embeddings.create(
       input = review['review'],
       model = 'text-embedding-3-small'
       
   )
   embedding = response.data[0].embedding
   processed_data.append({
       'values': embedding,
       'id': review['professor'],
       'metadata': {
           'review': review['review'],
           'subject': review['subject'],
           'stars' : review['stars']
       }
   })


In [ ]:
index = pc.Index('rag')

index.upsert(
    vectors=processed_data,
    namespace='ns1',
    
)

In [ ]:
index.describe_index_stats()